In [4]:
def advance_instruction_for_chef(session, chef_name: str):
    """
    Return a new Session in which:
      - the current instruction for chef_name is moved to past_instructions,
      - the next future instruction for chef_name (if any) is moved to current_instructions.
    Original session is not mutated.
    """
    # Deepcopy the lists to avoid modifying the original session
    past = session.past_instructions[:]
    current = []
    future = session.future_instructions[:]

    # Move the current instruction(s) for this chef to past
    for instr in session.current_instructions:
        if instr.chef.name == chef_name:
            past.append(instr)
        else:
            current.append(instr)

    # Find and move the next future instruction for this chef to current
    next_instr_index = None
    for idx, instr in enumerate(future):
        if instr.chef.name == chef_name:
            next_instr_index = idx
            break

    if next_instr_index is not None:
        current.append(future[next_instr_index])
        del future[next_instr_index]

    # Return a new Session
    return Session(
        past_instructions=past,
        current_instructions=current,
        future_instructions=future
    )

In [3]:
def join_chef(session, new_chef_name):
    """
    Return a new Session where the new chef is added and
    future_instructions are recompiled using the black-box
    compile_cooking_instruction function.
    
    Assumes:
      - session has a .recipe field (the full cooking recipe)
      - session.current_instructions and session.past_instructions are preserved
      - Chefs are inferred from all instructions in all lists
    """
    # Step 1: Get all current chefs (from instructions)
    def all_chefs_from_instructions(instruction_lists):
        chefs = []
        names = set()
        for instr_list in instruction_lists:
            for instr in instr_list:
                if instr.chef.name not in names:
                    chefs.append(instr.chef)
                    names.add(instr.chef.name)
        return chefs
    
    old_chefs = all_chefs_from_instructions([
        session.past_instructions,
        session.current_instructions,
        session.future_instructions
    ])
    
    # Step 2: Check if chef already exists
    if new_chef_name in [c.name for c in old_chefs]:
        # Chef already in session, just return a shallow copy of session
        return Session(
            past_instructions=session.past_instructions[:],
            current_instructions=session.current_instructions[:],
            future_instructions=session.future_instructions[:]
        )
    
    # Step 3: Add new Chef object (with empty personal_time_estimates, or customize as needed)
    new_chef = Chef(name=new_chef_name, personal_time_estimates={})
    all_chefs = old_chefs + [new_chef]
    
    # Step 4: Recompile only the future instructions
    # The black-box compile_cooking_instruction expects (recipe, chef_list)
    # The output is a list of Instructions for the remaining session steps
    new_future_instructions = compile_cooking_instruction(session.recipe, all_chefs)
    
    # Step 5: Build new session
    return Session(
        past_instructions=session.past_instructions[:],
        current_instructions=session.current_instructions[:],
        future_instructions=new_future_instructions
    )